In [1]:
# Import Dependencies
import os
import pandas as pd
import requests
import gmaps
import json
import matplotlib.pyplot as plt
from config import gkey

gmaps.configure(api_key=gkey)

In [2]:
citiesdf = pd.read_csv("cities_weather.csv", encoding="utf-8")
citiesdf.head()

,City,id,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,rikitea,4030556,PF,-23.120279,-134.969177,1607817600,100,80,75.54,12.44
1,hithadhoo,1282256,MV,-0.600000,73.083328,1607817600,41,77,82.22,17.11
2,bambous virieux,1106677,MU,-20.342779,57.757500,1607817600,55,78,75.38,16.62
3,lompoc,5367788,US,34.639149,-120.457939,1607817600,75,89,58.69,16.51
4,punta arenas,3874787,CL,-53.150002,-70.916672,1607817600,65,63,48.74,15.57


In [3]:
##
humidity = citiesdf["Humidity"].astype(float)
max_h = humidity.max()

sites =  citiesdf[["Lat","Lng"]]


In [4]:
fig = gmaps.figure()
heat = gmaps.heatmap_layer(sites, weights=humidity, dissipating=False, max_intensity=max_h, point_radius=2)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
nice_weather = citiesdf.loc[(citiesdf["Max Temp"] > 70) & (citiesdf["Max Temp"] < 80) & (citiesdf["Cloudiness"] == 0), :]
nice_weather.reset_index(inplace=True)
del nice_weather['index']
nice_weather.head(25)


,City,id,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,arlit,2447513,NE,18.736940,7.385280,1607817600,0,19,75.79,8.84
1,oum hadjer,2426370,TD,13.295400,19.696600,1607817600,0,17,77.29,8.34
2,cabo san lucas,3985710,MX,22.890881,-109.912376,1607817600,0,59,73.60,10.83
3,caravelas,3466980,BR,-17.712549,-39.248138,1607817600,0,79,75.81,7.29
4,puri,1259184,IN,19.799999,85.849998,1607817600,0,81,73.33,5.75
5,canavieiras,3467577,BR,-15.675000,-38.947220,1607817600,0,79,75.02,5.50
6,paratinga,3455029,BR,-12.690560,-43.184170,1607817600,0,52,77.29,6.96
7,diego de almagro,3892454,CL,-26.366671,-70.050003,1607817600,0,29,71.15,3.00
8,beloha,1067565,MG,-25.166670,45.049999,1607817600,0,73,72.28,12.77
9,edd,338345,ER,13.929720,41.692501,1607817600,0,67,75.79,9.26


In [6]:
# create a list to store hotel info
hotel = []

#For loop to get the hotel locations
for i in range(len(nice_weather)):
    lat = nice_weather.loc[i]['Lat']
    lon = nice_weather.loc[i]['Lng']
    ## set the parametrs for the api to locate hotels
    params = {
        "location": f"{lat},{lon}",
        "radius": 5000,
        "types": "hotel",
        "key": gkey        
    }
    # set the base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    request = requests.get(base_url, params=params)
    jsn = request.json()
    
    hotel.append(jsn['results'][0]['name'])
    
nice_weather["Hotel"] = hotel
nice_weather.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,id,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel
0,arlit,2447513,NE,18.736940,7.385280,1607817600,0,19,75.79,8.84,Arlit
1,oum hadjer,2426370,TD,13.295400,19.696600,1607817600,0,17,77.29,8.34,Oum Hadjer
2,cabo san lucas,3985710,MX,22.890881,-109.912376,1607817600,0,59,73.60,10.83,Cabo San Lucas
3,caravelas,3466980,BR,-17.712549,-39.248138,1607817600,0,79,75.81,7.29,Caravelas
4,puri,1259184,IN,19.799999,85.849998,1607817600,0,81,73.33,5.75,Puri


In [10]:
 # NOTE: Do not change any of the code in this cell
#popup info in this box is from the starter data
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in nice_weather.iterrows()]
locations = nice_weather[["Lat", "Lng"]]

In [13]:


markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))